# Базовая часть

In [178]:
reviews = ["""Телефон абонента: +7 916 000 00 01
Время: 10:34
Дата: 20.05.2017
Оценка: 1
Длительность разговора (мин.): 0:32
Комментарий: Общение с сотрудником вашей техподдержки мне не понравилось. Раньше всегда отвечала девушка с приятным голосом, а сегодня какой-то мужчина. Совсем никуда не годится, примите меры!!!""",
"""Телефон абонента: +7 916 000 00 02
Время: 9:20
Дата: 21.05.2017
Оценка: 5
Длительность разговора (мин.): 25:07
Комментарий: С моей проблемой мне помогли, большое спасибо!"""]

In [203]:
my_base = {}
my_sub_base = {}

for notes in reviews:
    if notes:
        splitted_line = notes.split('\n')[0].split(': ')
        curr_phone = splitted_line[1] # номер абонента из текущей записи
        my_sub_base = {} # временный вспомогательный словарь
        for line in notes.split('\n'):
            if line:
                note_line = line.split(': ')
                if note_line[0] <> "Телефон абонента": # для исключения дублирования хранения телефонных номеров в словаре, номер уже в индексе.
                    my_sub_base[note_line[0]] = note_line[1] # сформируем временный словарь из текущей записи
        my_base[curr_phone] = my_sub_base # заполним наш глобальный словарь очередной записью вида телефон-вложенный словарь со всеми полями

del(my_sub_base) # освободим память от временной переменной
    
my_base['+7 916 000 00 01']['Дата'] == '20.05.2017'


True

# Продвинутая Часть

Видится логичным сэкономит на хранении следующих данных:
1. Время выгодно хранить как один int. т.е. 00:00 = 0, 00:10 = 10, 01:10 = 110, 12:20 = 1220, 23:59 = 2359. Но, как реализовать эту логику в коде я подка не придумал
2. Дату так же логично хранить не как строку, а например, как количство дней с момента создания базы. То есть известно, что до 20.05.2017г заявок в базе нет и не будет - мы их еще не принимали от абонентов и не заносили в систему в принципе. Тогда для поступившей заявки 21.05.2017 года мы храним число 1, а абсолютную дату находим как результат сложения хранимого числа дней со священной датой создания базы 20.05.2017г. Если такое хранимое число (смещение в днях) поместить в двухбайтовую переменную (не знаю, как заставить python создать int размером в 2 байта, подскажете?), то его "хватит" на 65535/365~180 лет. Для этого надо нарисовать функцию, которая будет корректно увеличивать начальную дату с учетом разного числа дней в месяцах и високосного года, я пока опять же ее не придумал, так что буду хранить дату в форме импортируемой библиотеки datetime.data (не нашел, как определить ее размер в байтах, подскажете?).
3. Оценку точно логично хранить в 1 байте (точнее, в трех битах :), но применими int, который надеюсь выгоднее, чем строка из одного символа.
4. Разговор можно хранить в минутах (мобильные звонки больше 1 часа, как известно, автоматически обрываются сетью). То есть переводить 2:14 в 2*60+14 = 134 сек. Максимальное значение будет 59:59 = 59*60+59 = 3599. Опять же могло бы влезть в два байта. Но поместим в int.
5. Еще логичным видится повторяющиеся поля "Время", "Дата", "Оценка"... переименовать скажем в числа 1,2,3,4,5 и при дальнейшем использовании декодировать в строки текста, мое предположение, что это сэкономит память на хранении индексов. Не понятно, эти названия полей действительно в неизменном виде хранятся в памяти и занимают кучу места?
6. Еще можно (по аналогии с предыдущим пунктом) сэкономить на хранении телефонного номера, используя int для хранения 9 символов (без пробелов, плюса и семерки, повторяющихся в любых номерах наших абонентов) - скорее всего это 4 байта (сколько байт занимает тип int в python?). Тип str из 16 символов (длина строки "+7 916 000 00 01") точно занимает больше места, если верно мое предположение, что индекс в словаре хранится так неэкономно.

В итоге, что закодил:
1. время и дата поступления жалобы хранятся в полях одной переменной типа datetime 
2. оценка - int
3. длительность разговора храниться будет int в секундах (от 0 до 3599). Бонусом нарисовал код обратного перевода в формат ММ:СС

In [234]:
import datetime

In [233]:
a = "+7 916 000 00 01"
print type(a), len(a) 


d = datetime.date(2012, 12, 14)
print d, type(d)


<type 'str'> 16
2012-12-14 <type 'datetime.date'>


In [301]:
import datetime
my_base = {}
my_sub_base = {}

for notes in reviews:
    if notes:
        splitted_line = notes.split('\n')[0].split(': ')
        curr_phone = splitted_line[1] # номер абонента из текущей записи
        my_sub_base = {} # временный вспомогательный словарь, создается для ка
        for line in notes.split('\n'):
            if line:
                note_line = line.split(': ')
                if note_line[0] <> "Телефон абонента": # для исключения дублирования телефонных номеров в словаре
                    if note_line[0] == "Время":
                        time_hours = int(note_line[1].split(':')[0])
                        time_minutes = int(note_line[1].split(':')[1])                       
                    elif note_line[0] == "Дата":
                        time = datetime.datetime(int(note_line[1].split('.')[2]), int(note_line[1].split('.')[1]), int(note_line[1].split('.')[0]), time_hours, time_minutes)
                        my_sub_base["Дата и время"] = time
                    elif note_line[0] == "Оценка":
                        my_sub_base[note_line[0]] = int(note_line[1]) 
                    elif note_line[0] == "Длительность разговора (мин.)":
                        my_sub_base["Длительность разговора (сек.)"] = 60 * int(note_line[1].split(':')[0]) + int(note_line[1].split(':')[1])
                    else: 
                        my_sub_base[note_line[0]] = note_line[1]
        my_base[curr_phone]=my_sub_base #заполним наш глобальный словарь очередной записью вида телефон-вложенный словарь со всеми полями
        
del(my_sub_base) #освободим память от временных переменных
del(time)

print 'по номеру +7 916 000 00 01:'
print 'Дата и время: ',my_base['+7 916 000 00 01']['Дата и время'], type(my_base['+7 916 000 00 01']['Дата и время'])
print 'Оценка: ',my_base['+7 916 000 00 01']['Оценка'], type(my_base['+7 916 000 00 01']['Оценка'])
print 'Длительность разговора (сек.)',my_base['+7 916 000 00 01']['Длительность разговора (сек.)'], type(my_base['+7 916 000 00 01']['Длительность разговора (сек.)'])

# плюс нарисую код обратного перевода хранимой в секундах длительности разговора в начальный формат минуты:секунды:
time = my_base['+7 916 000 00 02']['Длительность разговора (сек.)']
print '\nпо номеру +7 916 000 00 02:'
print 'Длительность разговора (сек.): '+str(time)
time_minutes = str(time//60)
if len(time_minutes) < 2:
    time_minutes = "0"+time_minutes
time_sec = str(time%60)
if len(time_sec) < 2:
    time_sec = "0"+time_sec
print 'Длительность разговора (мин.): '+time_minutes+':'+time_sec

по номеру +7 916 000 00 01:
Дата и время:  2017-05-20 10:34:00 <type 'datetime.datetime'>
Оценка:  1 <type 'int'>
Длительность разговора (сек.) 32 <type 'int'>

по номеру +7 916 000 00 02:
Длительность разговора (сек.): 1507
Длительность разговора (мин.): 25:07


# Сложная часть

In [305]:
# tuple создан специально для защиты данных от изменения, но есть вариант сделать элементом кортежа изменяемый объект, 
# например список, и этот элемент получается менять:
test_tuple = ([1, 2], [3, 4]) 
test_tuple[0][0] = 5 
print test_tuple
# использовать tuple как индекс в словаре у меня не получилось. 


([5, 2], [3, 4])
